In [18]:
import json
import csv
import os
from glob import glob


In [8]:
folder_path = "3-preCorpus/"
os.makedirs(os.path.dirname(folder_path), exist_ok=True)

In [11]:
# Corpus base con definiciones clave
base_corpus = [
    ("¿Qué es un Feature Group?", "Un Feature Group es una colección de variables utilizadas en modelos analíticos dentro del Feature Store del banco."),
    ("¿Qué significa <vp>?", "<vp> representa la vicepresidencia a la que pertenece el Feature Group, puede ser 'per' para Banca Persona o 'emp' para Banca Empresa."),
    ("¿Qué significa <lat>?", "<lat> indica la periodicidad de carga del Feature Group, puede ser 'dia' (Diario), 'sem' (Semanal) o 'mes' (Mensual)."),
    ("¿Qué es el universo máximo en los datos del banco?", "El universo máximo define el conjunto de datos sobre el cual se aplican los Feature Groups: Población completa, Sistema financiero o Clientes del banco."),
]

with open(folder_path+"/3-base_corpus.json", "w", encoding="utf-8") as f:
    json.dump(base_corpus, f, ensure_ascii=False, indent=4)


In [13]:
def generate_corpus(data, corpus):    
    for row in data:
        dominio, repo, feature_group, bitbucket_fg, aws_tabla, universo, definicion = row
        
        # Pares de pregunta-respuesta
        corpus.extend([
            (f"¿Qué información contiene el feature group '{feature_group}'?", definicion),
            (f"¿Dónde se almacena el feature group '{feature_group}'?", f"Repositorio: {repo}, Bitbucket: {bitbucket_fg}, AWS Tabla: {aws_tabla}"),
            (f"¿Cuál es el universo de datos del feature group '{feature_group}'?", universo),
            (f"¿Cómo está relacionado el feature group '{feature_group}' con el dominio '{dominio}'?", f"Forma parte del dominio {dominio} y se encuentra en el repositorio {repo}."),
            (f"¿Cuál es el propósito del feature group '{feature_group}' en el contexto del banco?", f"{definicion} Este feature group pertenece al dominio {dominio} y su universo máximo es {universo}."),
        ])
        
        # Texto estructurado
        structured_text = (
            f"Dominio: {dominio}\n"
            f"Repositorio: {repo}\n"
            f"Feature Group: {feature_group}\n"
            f"Bitbucket Feature Group: {bitbucket_fg}\n"
            f"AWS Tabla: {aws_tabla}\n"
            f"Universo Máximo: {universo}\n"
            f"Definición: {definicion}\n"
        )
        corpus.append(("Texto estructurado", structured_text))
    
    return corpus


In [14]:


# Ejemplo de datos con la estructura mencionada
data = [
    ["Información personas ","ibk-fs-acoe-infpers","Datos sociodemográficos","<vp>-sociodem-<lat>","ibk_fs_group_<vp>_sociodem_<lat>"," Población completa","Información sociodemográfica, ingresos, edad, años de existencia y fuentes de ingresos."],
    ["Información personas ","ibk-fs-acoe-infpers","Propiedades ","<vp>-propiedad-<lat>","ibk_fs_group_<vp>_propiedad_<lat>"," Población completa","Información de propiedades muebles e inmuebles, tangibles e intangibles."],
    ["Información personas ","ibk-fs-acoe-infpers","Datos de contacto ","<vp>-contacto-<lat>","ibk_fs_group_<vp>_contacto_<lat>"," Población completa","Contactabilidad a nivel de personas para teléfonos o correos. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos.  "],
    ["Información personas ","ibk-fs-acoe-infpers","Contacto telefónico","ibk-ctostelf-<lat>","ibk_fs_group_ibk_ctostelf<lat>","Teléfonos de la población completa","Contactabilidad a nivel de teléfonos, no a nivel de personas. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos."],
    ["Información personas ","ibk-fs-acoe-infpers","Contacto digital","ibk-ctosdig-<lat>","ibk_fs_group_ibk_ctosdig_<lat>","Correos electrónicos de la población completa","Contactabilidad a nivel de correos electrónicos, no a nivel de personas. Incluye la cantidad o ratios de tipos de contactos, tales como el contacto efectivo, contacto no efectivo o sin contacto, para diferentes periodos.  "],
    ["Historial financiero ","ibk-fs-acoe-histfin","RCC ","<vp>-rcc-<lat>","ibk_fs_group_<vp>_rcc_<lat>","Sistema financiero","Información sobre los productos financieros, montos y variaciones en los montos de productos activos reportados en el sistema financiero."],
    ["Historial financiero ","ibk-fs-acoe-histfin","Riesgo financiero ","<vp>-rskfinac-<lat>","ibk_fs_group_<vp>_rskfinac_<lat>","Sistema financiero","Variables relacionadas con la evaluación del riesgo crediticio, días de mora o atraso en todo el sistema financiero (no exclusivamente Interbank) e información de centrales de riesgo de pagos."],
    ["Información cliente ","ibk-fs-acoe-infocli","Perfil del cliente ","<vp>-pflcom-<lat>","ibk_fs_group_<vp>_pflcom_<lat>","Sistema financiero","Descripciones sobre el perfil comercial de los potenciales clientes del banco, tales como el CLV del cliente, principalidad; y, segmentaciones o perfilamientos sobre el tipo de consumidor o nivel de digitalización, etc."],
    ["Información cliente ","ibk-fs-acoe-infocli","Bases de riesgos ibk","<vp>-rskibk-<lat>","ibk_fs_group_<vp>_rskibk_<lat>"," Sistema financiero","Información sobre los productos financieros aprobados por Interbank a clientes y no clientes, sin considerar si son ofertados o no. Incluye el monto ofertado, la tasa y la capacidad de endeudamiento máxima, conocida como CEM. Además, la evaluación del riesgo crediticio, días de mora o atraso en exclusivamente Interbank."],
    ["Información cliente ","ibk-fs-acoe-infocli","Productos ","<vp>-products-<lat>","ibk_fs_group_<vp>_products_<lat>"," Clientes del banco","Descripción de los montos, duración del préstamo, períodos de gracia, etc. Además de la cantidad de productos (activos y pasivos) y el saldo de productos."],
    ["Gestión del journey ","ibk-fs-acoe-journey","Journey ","<vp>-journey-<lat>","ibk_fs_group_<vp>_journey_<lat>"," Clientes del banco","Información sobre las etapas del ciclo de vida del cliente, tales como la adquisición, activación, desarrollo o retención."],
    ["Gestión del journey ","ibk-fs-acoe-journey","Churn ","<vp>-churn-<lat>","ibk_fs_group_<vp>_churn_<lat>"," Clientes del banco","Variables sobre los productos cancelados o bloqueados, tipos de cancelaciones, intentos de cancelaciones y canal de cancelación."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Oferta de productos ","<vp>-cmpprod-<lat>","ibk_fs_group_<vp>_cmpprod_<lat>"," Sistema financiero","Datos relacionados con la participación del cliente en campañas de oferta de productos, los productos ofertados, respuestas y canales."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Campañas de consumo ","<vp>-cmpcons-<lat>","ibk_fs_group_<vp>_cmpcons_<lat>"," Clientes del banco","Datos relacionados con la participación del cliente en campañas de consumo con crédito o débito, cantidad o ratios de promociones enviadas, tipos de promociones, canales de envío, nivel de recepción y respuestas."],
    ["Campañas y promociones ","ibk-fs-acoe-camp","Campañas de educación ","<vp>-cmpedu-<lat>","ibk_fs_group_<vp>_cmpedu_<lat>"," Clientes del banco","Datos relacionados con la participación del cliente en campañas de educación de uso de productos o app, cantidad o ratios de comunicación enviadas, canales de envío, nivel de recepción y respuestas."],
    ["Transacciones financieras ","ibk-fs-acoe-trx","Transacciones ","<vp>-trx-<lat>","ibk_fs_group_<vp>_trx_<lat>"," Clientes del banco","Información sobre las transacciones realizadas por los clientes, montos transaccionados, medios y canales por donde se transacciono."],
    ["Transacciones financieras ","ibk-fs-acoe-trx","Consumos ","<vp>-consumos-<lat>","ibk_fs_group_<vp>_consumos_<lat>"," Clientes del banco","Información de los consumos con tarjetas realizadas por los clientes, incluyendo el rubro de consumo y medio o canal."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias de en tiendas ","<vp>-exptienda-<lat>","ibk_fs_group_<vp>_exptienda_<lat>"," Sistema financiero","Información de la interacción de los clientes en las tiendas, sus arribos, NPS, tiempos de espera, motivos de arribos, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias televentas ","<vp>-exptlv-<lat>","ibk_fs_group_<vp>_exptlv_<lat>"," Sistema financiero","Información de la interacción de los clientes y potenciales clientes en televentas, motivos de no aceptación, cantidad de llamadas, productos ofertados por canal, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias banca telefónica ","<vp>-bnktelf-<lat>","ibk_fs_group_<vp>_bnktelf_<lat>"," Sistema financiero","Información de la interacción de los clientes o potenciales clientes en banca telefónica, motivos de llamada, cantidad de llamadas, tiempos de respuesta, cadencia, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencias chats ","<vp>-expchats-<lat>","ibk_fs_group_<vp>_expchats_<lat>"," Sistema financiero","Información de la comunicación de los clientes o potenciales clientes con los chatbots u operadores de chats (whatsapp, Facebook, etc). Incluyendo los motivos de la comunicación, resolutividad, tiempos de respuesta, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencia APP ","<vp>-expapp-<lat>","ibk_fs_group_<vp>_expapp_<lat>"," Clientes del banco","Información de experiencia de los clientes con las apps del banco, incluyendo tiempos de navegación, clicks, número de consultas en la app, etc."],
    ["Experiencia cliente ","ibk-fs-acoe-exp","Experiencia digital ","<vp>-expdig-<lat>","ibk_fs_group_<vp>_expdig_<lat>"," Sistema financiero","Información sobre otras variables digitales de los clientes que no tengan que ver con transacciones o las apps del banco, incluyendo uso de canales físicos-digitales, como ATMs y variables de la web del banco."],
    ["Cobranzas ","ibk-fs-acoe-cob","Comportamiento de pagos","<vp>-pagos-<lat>","ibk_fs_group_<vp>_pagos_<lat>"," Clientes del banco","Historial de pagos por tipo de producto dentro de Interbank, incluyendo días de mora y deuda acumulada."],
    ["Cobranzas ","ibk-fs-acoe-cob","Medios de cobranza","<vp>-cobros-<lat>","ibk_fs_group_<vp>_cobros_<lat>"," Clientes del banco","Medios de comunicación para las cobranzas, cantidad de llamadas, cantidad de llamadas rechazadas, etc."],
    ["Variables INTERCORP","ibk-fs-acoe-intercorp","Intercorp transformada","ibk-itcx-<lat>","ibk_fs_group_ibk_itcx_<lat>","Población completa","Información sobre las variables transformadas de intercorp, y en que modelos se utilizan."]                            
]

# Generar el corpus
generated_corpus = generate_corpus(data, base_corpus.copy())

# Mostrar ejemplos
top_5_samples = generated_corpus[:15]
for sample in top_5_samples:
    print(sample)

# Guardar el corpus en un archivo JSON (reemplaza si existe)
with open(folder_path+"/3-corpus_acoe.json", "w", encoding="utf-8") as f:
    json.dump(generated_corpus, f, ensure_ascii=False, indent=4)

('¿Qué es un Feature Group?', 'Un Feature Group es una colección de variables utilizadas en modelos analíticos dentro del Feature Store del banco.')
('¿Qué significa <vp>?', "<vp> representa la vicepresidencia a la que pertenece el Feature Group, puede ser 'per' para Banca Persona o 'emp' para Banca Empresa.")
('¿Qué significa <lat>?', "<lat> indica la periodicidad de carga del Feature Group, puede ser 'dia' (Diario), 'sem' (Semanal) o 'mes' (Mensual).")
('¿Qué es el universo máximo en los datos del banco?', 'El universo máximo define el conjunto de datos sobre el cual se aplican los Feature Groups: Población completa, Sistema financiero o Clientes del banco.')
("¿Qué información contiene el feature group 'Datos sociodemográficos'?", 'Información sociodemográfica, ingresos, edad, años de existencia y fuentes de ingresos.')
("¿Dónde se almacena el feature group 'Datos sociodemográficos'?", 'Repositorio: ibk-fs-acoe-infpers, Bitbucket: <vp>-sociodem-<lat>, AWS Tabla: ibk_fs_group_<vp>_so

In [15]:

def load_csv_to_array(filename):
    data = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)  # Ignorar la primera fila (headers)
        for row in reader:
            processed_row = [col if col.strip() else "nan" for col in row[:5]]
            data.append(processed_row)
    return data

data_csv = load_csv_to_array(folder_path+"/meta_fs - corpus.csv")

In [16]:
def generate_corpus_csv(data):
    corpus = []
    
    for row in data:
        variable, descripcion, tablas_fuente, dominio, p_featuregroup = row
        
        # Pares de pregunta-respuesta
        corpus.extend([
            (f"¿Qué significa la variable '{variable}'?", descripcion),
            (f"¿A qué tabla fuente pertenece la variable '{variable}'?", f"Se encuentra en la tabla fuente: {tablas_fuente}"),
            (f"¿A qué dominio pertenece la variable '{variable}'?", f"Forma parte del dominio: {dominio}"),
            (f"¿En qué feature group se encuentra la variable '{variable}'?", f"Se encuentra en el feature group: {p_featuregroup}"),
            (f"¿Cuál es el propósito de la variable '{variable}'?", f"{descripcion} Esta variable pertenece al dominio {dominio} y está en la tabla {tablas_fuente} dentro del feature group {p_featuregroup}.")
        ])
        
        # Texto estructurado
        structured_text = (
            f"Variable: {variable}\n"
            f"Descripción: {descripcion}\n"
            f"Tabla Fuente: {tablas_fuente}\n"
            f"Dominio: {dominio}\n"
            f"Feature Group: {p_featuregroup}\n"
        )
        corpus.append(("Texto estructurado", structured_text))
    
    return corpus


In [17]:
generated_corpus_csv = generate_corpus_csv(data_csv)

# Mostrar ejemplos
top_5_samples = generated_corpus_csv[:15]
for sample in top_5_samples:
    print(sample)

# Guardar el corpus en un archivo JSON (reemplaza si existe)
with open(folder_path+"/3-corpus_csv.json", "w", encoding="utf-8") as f:
    json.dump(generated_corpus_csv, f, ensure_ascii=False, indent=4)

("¿Qué significa la variable 'flg_clicambfch09m'?", 'Indicador si el cliente solicito un cambio de fecha en los 9 últimos meses')
("¿A qué tabla fuente pertenece la variable 'flg_clicambfch09m'?", 'Se encuentra en la tabla fuente: t_agg_cambiofecha_hist_rsk')
("¿A qué dominio pertenece la variable 'flg_clicambfch09m'?", 'Forma parte del dominio: ibk-fs-acoe-journey')
("¿En qué feature group se encuentra la variable 'flg_clicambfch09m'?", 'Se encuentra en el feature group: ibk-fs-group-emp-journey-mes')
("¿Cuál es el propósito de la variable 'flg_clicambfch09m'?", 'Indicador si el cliente solicito un cambio de fecha en los 9 últimos meses Esta variable pertenece al dominio ibk-fs-acoe-journey y está en la tabla t_agg_cambiofecha_hist_rsk dentro del feature group ibk-fs-group-emp-journey-mes.')
('Texto estructurado', 'Variable: flg_clicambfch09m\nDescripción: Indicador si el cliente solicito un cambio de fecha en los 9 últimos meses\nTabla Fuente: t_agg_cambiofecha_hist_rsk\nDominio: ibk

In [20]:
json_files = glob(os.path.join(folder_path, "*.json"))

preguntas_respuestas = []
texto_estructurado = []

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
        for item in data:
            if isinstance(item, list) and item[0] == "Texto estructurado":
                texto_estructurado.append(item[1])
            elif isinstance(item, list) and len(item) == 2:
                preguntas_respuestas.append(item) 

# Guardar los archivos JSON de salida
with open("3-corpus_preguntas_respuestas.json", "w", encoding="utf-8") as f:
    json.dump(preguntas_respuestas, f, ensure_ascii=False, indent=4)

with open("3-corpus_texto_estructurado.json", "w", encoding="utf-8") as f:
    json.dump(texto_estructurado, f, ensure_ascii=False, indent=4)

print("Archivos generados correctamente.")


Archivos generados correctamente.


In [ ]:
#TEST corpus v2
import json
import random

# Cargar el corpus de entrenamiento
with open("3-corpus.json", "r", encoding="utf-8") as file:
    corpus = json.load(file)

# Función para generar preguntas reformuladas
def generar_pregunta_simulada(pregunta):
    variaciones = [
        f"¿Podrías explicarme {pregunta[2:].lower()}?", 
        f"Me gustaría saber {pregunta[2:].lower()}.",
        f"Tengo dudas sobre {pregunta[2:].lower()}, ¿puedes ayudarme?",
        f"¿Cómo se define {pregunta[2:].lower()}?",
        f"¿Qué significa exactamente {pregunta[2:].lower()}?"
    ]
    return random.choice(variaciones)

# Filtrar solo pares de pregunta-respuesta válidos
preguntas_respuestas = [pair for pair in corpus if pair[0].startswith("¿")]

# Determinar el tamaño del conjunto de prueba (20%)
test_set_size = int(len(preguntas_respuestas) * 0.2)

# Seleccionar una muestra aleatoria y reformular las preguntas
test_set_simulated = [
    [generar_pregunta_simulada(q), a] for q, a in random.sample(preguntas_respuestas, test_set_size)
]

# Guardar el conjunto de prueba generado
with open("3-test_set_v2.json", "w", encoding="utf-8") as file:
    json.dump(test_set_simulated, file, ensure_ascii=False, indent=4)

print("Conjunto de prueba generado: 3-test_set_v2.json")


Conjunto de prueba generado: 3-test_set_v2.json
